Face web scraping do forecasts de DXY da pagina  https://www.forecasts.org/dollar-forecast.htm

gera df con os títulos ao consultar (symbol , url)

In [1]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("sqforecasts.db")

# Criar a consulta SQL
query = "SELECT symbol, url FROM tbtitulos"

# Criar o DataFrame com os dados da consulta
dftitulos = pd.read_sql(query, conn)

# Fechar a conexão
conn.close()

# Exibir o DataFrame
display(dftitulos)

,symbol,url
0,DXY,https://www.forecasts.org/dollar-forecast.htm
1,US10Y,https://www.forecasts.org/10yrT.htm
2,USIR,https://www.forecasts.org/inflation.htm
3,SPX,https://www.forecasts.org/stpoor.htm


In [3]:
url= str(dftitulos.iloc[0]['url'])
print(url)

https://www.forecasts.org/dollar-forecast.htm


In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL da página de previsão do U.S. Dollar Index


# Função para obter os dados do índice
def get_fore_dados(url,symbol):
    # Fazer a requisição HTTP para a página
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve erro na requisição

    # Criar o objeto BeautifulSoup para processar o HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Localizar a tabela pelo ID (caso tenha um identificador)
    table = soup.find("table", {"id": "forecast"})
    
    # Extrair todas as linhas da tabela
    rows = table.find_all("tr")[1:]  # Ignora o cabeçalho

    # Criar lista para armazenar os dados extraídos
    data_list = []
    
    for row in rows:
        cols = row.find_all("td")
        month = cols[1].text.strip()
        date = pd.to_datetime(month)  # Convertendo a string para uma data válida
        forecast_value = float(cols[2].text.strip())
        avg_error = float(cols[3].text.replace("±", "").strip())

        data_list.append([month, date, forecast_value, avg_error])

    # Criar DataFrame
    df_forecasts_scrap = pd.DataFrame(data_list, columns=["Month", "Date", "Forecast Value", "Avg Error"])

    # --- Capturar a data de atualização ---
    updated_text = soup.find("time").text.strip()
    updated_date = pd.to_datetime(updated_text.replace("Updated: ", ""))  # Converter para formato datetime

    return df_forecasts_scrap, updated_date, symbol


url = str(dftitulos.iloc[2]['url'])
symbol = str(dftitulos.iloc[2]['symbol'])

# Chamar a função para obter os dados
df_forecasts_scrap, last_updated, symbol = get_fore_dados(url,symbol)

# Exibir os dados
display(symbol)
display(f"Última atualização: {last_updated}")
display(df_forecasts_scrap)


'USIR'

'Última atualização: 2025-04-11 00:00:00'

,Month,Date,Forecast Value,Avg Error
0,Mar 2025,2025-03-01,2.39,0.000
1,Apr 2025,2025-04-01,2.09,0.082
2,May 2025,2025-05-01,1.81,0.100
3,Jun 2025,2025-06-01,1.71,0.120
4,Jul 2025,2025-07-01,1.49,0.130
5,Aug 2025,2025-08-01,1.58,0.140
6,Sep 2025,2025-09-01,1.57,0.140
7,Oct 2025,2025-10-01,1.64,0.150
8,Nov 2025,2025-11-01,1.66,0.150


In [7]:
import sqlite3

# Conectar ao banco de dados
conn = sqlite3.connect('sqforecasts.db')
cursor = conn.cursor()

# Verificar se já existe um registro com os valores na coluna symbol
cursor.execute("SELECT id_forecasts FROM tbforecasts WHERE symbol = ? AND data = ?", (symbol, str(last_updated)))
result = cursor.fetchone()

if result:
    print("A atualização já existe.")
    id_forecasts_insert = result[0]
else:
    # Inserir novo registro com valores corretos
    cursor.execute("INSERT INTO tbforecasts (symbol, data) VALUES (?, ?)", (symbol, str(last_updated)))
    conn.commit()
    
    # Obter o ID do registro recém-inserido
    id_forecasts_insert = cursor.lastrowid
    print(f"Novo registro inserido com id_forecasts: {id_forecasts_insert}")

# Fechar conexão
conn.close()

Novo registro inserido com id_forecasts: 2


In [13]:
import sqlite3
import pandas as pd

# Nome do banco de dados SQLite
db_path = "sqforecasts.db"

df_forecasts_scrap = df_forecasts_scrap.drop(columns=["Month"], errors="ignore")  # Remove 'Month' se existir

# Renomeando colunas para coincidir com a tabela no banco de dados
df_forecasts_scrap.rename(columns={"Date": "data", "Forecast Value": "value", "Avg Error": "error"}, inplace=True)

# Adicionando a coluna id_forecasts
df_forecasts_scrap["id_forecasts"] = id_forecasts_insert

# Conectar ao banco SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Inserindo os dados na tabela
df_forecasts_scrap.to_sql("tbforecastsdados", con=conn, if_exists="append", index=False)

# Fechar conexão
conn.close()

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


In [11]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("sqforecasts.db")
cursor = conn.cursor()


# Obter o ID gerado automaticamente
id_forecasts = cursor.lastrowid

print(id_forecasts)

# --- Passo 2: Inserir os dados do df_forecasts_scrap na tabela tbforecastsdados ---
for _, row in df_forecasts_scrap.iterrows():
    cursor.execute("""
        INSERT INTO tbforecastsdados (id_forecasts, data, value, error) 
        VALUES (?, ?, ?, ?)
    """, (id_forecasts, str(row["Date"]), row["Forecast Value"], row["Avg Error"]))

print(f"Registros adicionados na tabela tbforecastsdados para id_forecasts = {id_forecasts}")

# Confirmar as alterações e fechar conexão
conn.commit()
conn.close()

print("Dados exportados com sucesso para o banco de dados SQLite!")

None


IntegrityError: NOT NULL constraint failed: tbforecastsdados.id_forecasts

Bono 10 y

In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os

# URL da página de previsão dos Treasury Yields
url = "https://www.forecasts.org/10yrT.htm"

# Arquivo para armazenar a última atualização
UPDATE_FILE = "last_update.txt"

# Função para obter os dados da tabela
def get_treasury_data():
    # Fazer requisição HTTP para obter o conteúdo da página
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve erro na requisição

    # Processar o HTML com BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Localizar a tabela pelo ID "forecast"
    table = soup.find("table", {"id": "forecast"})
    rows = table.find_all("tr")[1:]  # Ignorar cabeçalho

    # Criar lista para armazenar os dados extraídos
    data_list = []

    for row in rows:
        cols = row.find_all("td")
        month = cols[1].text.strip()
        date = pd.to_datetime(month)  # Converter para formato datetime
        forecast_value = float(cols[2].text.strip())
        avg_error = float(cols[3].text.replace("±", "").strip())

        data_list.append([month, date, forecast_value, avg_error])

    # Criar DataFrame
    df_treasury = pd.DataFrame(data_list, columns=["Month", "Date", "Forecast Value", "Avg Error"])

    # Capturar a data de atualização
    updated_text = soup.find("time").text.strip()
    updated_date = pd.to_datetime(updated_text.replace("Updated: ", ""))  # Converter para formato datetime

    return df_treasury, updated_date

# Função para verificar e salvar atualização
def check_for_update():
    df_treasury, last_updated = get_treasury_data()

    # Verificar se há um arquivo de última atualização salvo
    if os.path.exists(UPDATE_FILE):
        with open(UPDATE_FILE, "r") as file:
            last_saved_date = pd.to_datetime(file.read().strip())

        # Se houver uma nova atualização, salvar novo CSV e atualizar o arquivo de controle
        if last_updated > last_saved_date:
            print(f"Nova atualização encontrada em {last_updated}! Salvando dados...")
            df_treasury.to_csv("treasury_forecast.csv", index=False)

            with open(UPDATE_FILE, "w") as file:
                file.write(str(last_updated))
        else:
            print("Nenhuma atualização nova encontrada.")
    else:
        # Primeira execução - salvar a data
        with open(UPDATE_FILE, "w") as file:
            file.write(str(last_updated))
        print("Primeira execução do script. Criando CSV inicial.")
        df_treasury.to_csv("treasury_forecast.csv", index=False)

# Executar a verificação automática
check_for_update()

Nenhuma atualização nova encontrada.
